In [2]:
!nvidia-smi

Sun May  1 02:55:20 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 510.54       Driver Version: 510.54       CUDA Version: 11.6     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA A40          On   | 00000000:01:00.0 Off |                  Off |
|  0%   57C    P0   289W / 300W |  30967MiB / 49140MiB |     30%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
|   1  NVIDIA A40          On   | 00000000:23:00.0 Off |                  Off |
|  0%   

In [3]:
import pandas as pd
import numpy as np
import random
from tqdm.auto import tqdm
import time
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader

from transformers import BertTokenizerFast as BertTokenizer, BertModel, AdamW, get_linear_schedule_with_warmup
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, multilabel_confusion_matrix
from torch.cuda.amp import autocast as autocast
from torch.cuda.amp import GradScaler as GradScaler
import os


torch.cuda.set_device(1)
device = torch.device("cuda:1" if torch.cuda.is_available() else "cpu")

BERT_MODEL_NAME = 'bert-base-uncased'
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

EPOCHS = 1
MAX_TOKEN_COUNT = 512
BATCH_SIZE = 32

In [4]:
def setup_seed(seed):
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    np.random.seed(seed)
    random.seed(seed)
    torch.backends.cudnn.deterministic = True
# 设置随机数种子
setup_seed(20)

In [5]:
%%time 
train_df=pd.read_feather("Hierarchical major.feather")
val_df=pd.read_feather("./autodl-nas/USPTO-2M_Validation.feather")

# LABEL_COLUMNS=train_df.columns[11:]
# LABEL_COLUMNS

CPU times: user 37.2 s, sys: 4min 14s, total: 4min 51s
Wall time: 1min 9s


In [6]:
LABEL_COLUMNS_3 = train_df.columns[11:-139]
LABEL_COLUMNS_2 =train_df.columns[-129:]
LABEL_COLUMNS_1 = train_df.columns[-138:-129]

In [7]:
class PatentDataset(Dataset):
    """
    Pass pandas dataframe, and tokeizer along with the max token length[128 default]
    
    Example: 
    -------
    train_dataset = ToxicCommentsDataset(
      train_df,
      tokenizer,
      max_token_len=MAX_TOKEN_COUNT
    )

    sample_item = train_dataset[0]
    
    """
    
    
    def __init__(
        self,
        data: pd.DataFrame,
        tokenizer: BertTokenizer,
        max_token_len: int = 512,
        test= False
    ):
        self.data = data
        self.tokenizer = tokenizer
        self.max_token_len = max_token_len
        self.valid = test
        
    
    def __len__(self):
        return len(self.data)
    
    
    def __getitem__(self, index: int):
        data_row = self.data.iloc[index]

        if not self.valid:
            comment_text = data_row['major claim']
            label_3 = data_row[LABEL_COLUMNS_3]
            label_2 = data_row[LABEL_COLUMNS_2]
            label_1 = data_row[LABEL_COLUMNS_1]
            
        else:
            comment_text = "".join(data_row['claims'])
            label_3 = data_row[LABEL_COLUMNS_3]
        
        encoding = self.tokenizer.encode_plus(
            comment_text,
            max_length=self.max_token_len,
            padding="max_length",
            truncation=True,
            add_special_tokens=True, # [CLS] & [SEP]
            return_token_type_ids=False,
            return_attention_mask=True, #attention_mask
            return_tensors='pt',
        )
        
        if not self.valid:
            return dict(
            comment_text=comment_text,
            input_ids = encoding["input_ids"].flatten(),
            attention_mask=encoding["attention_mask"].flatten(),
            label_1=torch.FloatTensor(label_1),
            label_2=torch.FloatTensor(label_2),
            label_3=torch.FloatTensor(label_3),
                
        )
        else:
            return dict(
                comment_text=comment_text,
                input_ids = encoding["input_ids"].flatten(),
                attention_mask=encoding["attention_mask"].flatten(),
                label_3=torch.FloatTensor(label_3)
            )
        

In [25]:
train_dataset = PatentDataset(
  train_df,
  tokenizer,
  max_token_len=MAX_TOKEN_COUNT
)

val_dataset = PatentDataset(
  val_df,
  tokenizer,
  max_token_len=MAX_TOKEN_COUNT,
  test=True
)

In [26]:
train_dataloader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True,drop_last = True,pin_memory=True)
val_dataloader = DataLoader(val_dataset, batch_size=BATCH_SIZE, shuffle=False, drop_last = True,pin_memory=True)

In [10]:
Bert_model = BertModel.from_pretrained(BERT_MODEL_NAME,return_dict = True).to(device)

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.transform.dense.bias', 'cls.seq_relationship.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [11]:
Tags_df = pd.read_feather("Tags.feather")

In [12]:
class Tags_datasets(Dataset):
    
    def __init__(
        self,
        data: pd.DataFrame,
        tokenizer: BertTokenizer,
        max_token_len: int = 512
    ):
        self.data = data
        self.tokenizer = tokenizer
        self.max_token_len = max_token_len
    
    def __len__(self):
        return len(self.data)
    
        
    def __getitem__(self, index: int):
        data_row = self.data.iloc[index]
        labels = data_row.Label
        tags = data_row.Tags
        
        encoding = self.tokenizer.encode_plus(
            tags,
            max_length=self.max_token_len,
            padding="max_length",
            truncation=True,
            return_tensors='pt',
        )
        
        return dict(
            index=index,
            label = labels,
            input_ids=encoding['input_ids'].flatten(),
            attention_mask = encoding['attention_mask'].flatten()
        )

In [13]:
Tags_dataset = Tags_datasets(data = Tags_df,tokenizer = tokenizer, max_token_len = MAX_TOKEN_COUNT)
Tags_dataloader = DataLoader(Tags_dataset, batch_size = 1)

In [14]:
tags_embeddings = {}

for step,batch in tqdm(enumerate(Tags_dataloader),total = len(Tags_dataloader)):
    label = batch['label'][0]
    input_ids = batch['input_ids'].to(device)
    attention_mask = batch['attention_mask'].to(device)
    
    with torch.no_grad():
        output = Bert_model(input_ids,attention_mask = attention_mask)
    tag_embeddings = output.pooler_output.detach().cpu()
    tags_embeddings[label] = tag_embeddings




  0%|          | 0/674 [00:00<?, ?it/s]

In [15]:
tags_embedding = np.concatenate([tags_embeddings[key] for key in LABEL_COLUMNS_3],axis = 0)
tags_embedding = torch.tensor(tags_embedding)

In [16]:
tags_embedding = tags_embedding.to(device)

In [17]:
# Contrastive learning loss funcation

class ConLoss(nn.Module):
    
    def __init__(self, temperature = 0.1):
        super().__init__()
        self.temperature = temperature
        
    def forward(self, embeddings, label_1,label_2,label_3):
       
        # embeddings (batch_size,bert_hidden_layer) , labels (batch_size,num_classes)
        similarities = F.cosine_similarity(embeddings.unsqueeze(1),embeddings.unsqueeze(0),dim=2)/self.temperature
#         print(similarities)
        
        # mask
        logits_mask = ~torch.eye(BATCH_SIZE,dtype=torch.bool).to(device)
        labels_mask = ~torch.eye(BATCH_SIZE,dtype=torch.bool).to(device)

        exp_logits = torch.exp(similarities) * logits_mask
        loss = torch.log(exp_logits.sum(1,keepdim=True)) - similarities
        
        # labels 
        weight_1 = torch.matmul(label_1,label_1.T)
        weight_2 = torch.matmul(label_2,label_2.T)
        weight_3 = torch.matmul(label_3,label_3.T)

        weight_1 = weight_1 / torch.diag(weight_1)
        weight_1 = weight_1 * labels_mask
        weight_1 = torch.where(weight_1!=1,torch.zeros_like(weight_1).to(device),weight_1)
        weight_1 = weight_1/torch.sum(weight_1)
        
        weight_2 = weight_2 / torch.diag(weight_2)
        weight_2 = weight_2 * labels_mask
        weight_2 = torch.where(weight_2!=1,torch.zeros_like(weight_2).to(device),weight_2)
        weight_2 = weight_2/torch.sum(weight_2)
        
        weight_3 = weight_3 / torch.diag(weight_3)
        weight_3 = weight_3 * labels_mask
        weight_3 = torch.where(weight_3!=1,torch.zeros_like(weight_3).to(device),weight_3)
        weight_3 = weight_3/torch.sum(weight_3)
        
        weight = weight_1*0.2 + weight_2*0.3 + weight_3*0.5
        
        loss = weight * loss
        loss = torch.mean(loss)
        return loss
        


In [18]:
class BertNetwork(nn.Module):

    def __init__(self, n_classes: int,labels_embeddings: torch.Tensor, n_training_steps=None, n_warmup_steps=None):
        super().__init__()
        self.bert = BertModel.from_pretrained(BERT_MODEL_NAME, return_dict=True) #load the pretrained bert model
        self.classifier = nn.Linear(self.bert.config.hidden_size, n_classes) # add a linear layer to the bert
        self.n_training_steps = n_training_steps
        self.n_warmup_steps = n_warmup_steps
        self.criterion = nn.BCEWithLogitsLoss()
        self.Weight_1 = nn.Linear(n_classes,1)
        self.dropout = nn.Dropout(self.bert.config.hidden_dropout_prob)
        self.batch_labels_embeddings = labels_embeddings.expand(BATCH_SIZE,n_classes,self.bert.config.hidden_size)
        self.batch_labels_embeddings = self.batch_labels_embeddings.transpose(1, 2) # batch_size * 768 * 664
        self.batch_labels_embeddings.requires_grad = False
        
    def forward(self, input_ids, attention_mask):
        outputs = self.bert(input_ids, attention_mask=attention_mask)
        outputs = outputs.last_hidden_state
        
        # label-wise attention
        att = torch.bmm(outputs,self.batch_labels_embeddings)
        att = F.softmax(att,dim=1)
        joint_labels_output = torch.bmm(outputs.transpose(1,2),att)
        joint_labels_output = F.relu(self.Weight_1(joint_labels_output).squeeze(2))

        return joint_labels_output

In [19]:
class Classifier(nn.Module):
    
    def __init__(self, n_classes: int ):
        super().__init__()
        self.classifier = nn.Linear(768,n_classes)
        self.dropout = nn.Dropout(0.10)
        self.criterion = nn.BCEWithLogitsLoss()
        
    def forward(self,embeddings,labels = None):
        
        output = self.classifier(self.dropout(embeddings))
        
        if labels is not None:
            loss = self.criterion(output,labels)
            output = (loss,output)
        
        return output

In [20]:
model = BertNetwork(len(LABEL_COLUMNS_3),tags_embedding).to(device)
Conloss =  ConLoss().to(device)
classifier = Classifier(len(LABEL_COLUMNS_3)).to(device)
N_EPOCHS = EPOCHS

steps_per_epoch=len(train_df) // BATCH_SIZE
total_training_steps = steps_per_epoch * N_EPOCHS
warmup_steps = total_training_steps // 10
warmup_steps, total_training_steps

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.transform.dense.bias', 'cls.seq_relationship.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


(6248, 62484)

In [21]:
# optimizer = AdamW(model.parameters(), lr=2e-4)

optimizer = AdamW([
                {'params': model.parameters()},
                {'params': classifier.parameters()}],
                lr=5e-5
)

scheduler = get_linear_schedule_with_warmup(
    optimizer,
    num_warmup_steps=warmup_steps,
    num_training_steps=total_training_steps
)

/root/miniconda3/lib/python3.8/site-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


In [22]:
from sklearn.metrics import roc_auc_score
from sklearn.metrics import precision_score

# function for evaluating the model
def evaluate(mydataloader):

    print("\nEvaluating...")
    #t0 = time.time()
    # deactivate dropout layers
    model.eval()
    classifier.eval()
    total_loss, total_accuracy = 0, 0

    # empty list to save the model predictions
    total_preds = []
    total_labels = []

    # iterate over batches
    for step,batch in tqdm(enumerate(mydataloader),total=len(mydataloader),desc='Eval'):

        input_ids = batch["input_ids"].to(device)
        attention_mask = batch["attention_mask"].to(device)
#         label_1 = batch['label_1'].to(device)
#         label_2 = batch['label_2'].to(device)
        label_3 = batch['label_3'].to(device)
        # deactivate autograd
        with autocast():
            with torch.no_grad():
                outputs = model(input_ids, attention_mask)
#                 loss1 = Conloss(outputs,label_1=label_1,label_2=label_2,label_3=label_3)
                loss2,outputs = classifier(outputs,label_3)
                if step%1000 ==0:
                    print(f"loss contrastive :loss1  loss classifier: {loss2}")
                outputs = torch.sigmoid(outputs)

#                 loss = loss1 +loss2
                loss = loss2
                total_loss = total_loss + loss.float().item()

                outputs = outputs.detach().float().cpu().numpy()
                labels = label_3.detach().float().cpu().numpy()
                total_preds.append(outputs)
                total_labels.append(labels)


    # compute the validation loss of the epoch
    avg_loss = total_loss / len(mydataloader)

    total_preds  = np.concatenate(total_preds, axis=0)
    total_labels = np.concatenate(total_labels, axis=0)
    model.train()
    print(f"Evaluate loss {total_loss / len(mydataloader)}")
    return avg_loss, total_preds, total_labels

In [23]:
# function to train the model
def train():
    
    now=time.strftime('%Y-%m-%d %H:%M:%S', time.localtime())
    best_valid_loss = float('inf')
    model.train()
    classifier.train()
    total_loss, total_accuracy = 0, 0
    avg_loss = 0
    scaler = GradScaler()

    # empty list to save model predictions
    total_preds=[]
    # iterate over batches
    for step,batch in tqdm(enumerate(train_dataloader),total=len(train_dataloader),desc="Train"):
        
        if step%10000 == 0 and step!=0:
            valid_loss,_,_ = evaluate(val_dataloader)
            if valid_loss < best_valid_loss:
                best_valid_loss = valid_loss
                torch.save(model.state_dict(), f"./model/CoPatE_lr_5e-5_{now}.pt")
        input_ids = batch["input_ids"].to(device)
        attention_mask = batch["attention_mask"].to(device)
        label_1 = batch['label_1'].to(device)
        label_2 = batch['label_2'].to(device)
        label_3 = batch['label_3'].to(device)  
        if step > 0.8*len(train_dataloader):
            with autocast():
                outputs = model(input_ids, attention_mask)
                loss1 = Conloss(outputs,label_1=label_1,label_2=label_2,label_3=label_3)
                loss2,_ = classifier(outputs,label_3)
                loss = loss1 + loss2

            optimizer.zero_grad()
            if step%100 == 0:
                print(f"STEP {step}: loss contrastive :{loss1}  loss classifier: {loss2}")
        else:
            with autocast():
                outputs = model(input_ids, attention_mask)
                loss,_ = classifier(outputs,label_3)
            optimizer.zero_grad()
            if step%100 == 0:
                print(f"STEP {step}: loss classifier: {loss}")
        # add on to the total loss
        total_loss = total_loss + loss.item()
        scaler.scale(loss).backward()

        scaler.unscale_(optimizer)
        # clip the the gradients to 1.0. It helps in preventing the exploding gradient problem
        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)

        scaler.step(optimizer)
        scheduler.step()
        scaler.update()
        # model predictions are stored on GPU. So, push it to CPU
        outputs=outputs.detach().float().cpu().numpy()

        # append the model predictions
        total_preds.append(outputs)

    # compute the training loss of the epoch
    avg_loss = total_loss / len(train_dataloader)
    print(f"{step}: {avg_loss}")
  

    # predictions are in the form of (no. of batches, size of batch, no. of classes).
    # reshape the predictions in form of (number of samples, no. of classes)
    total_preds  = np.concatenate(total_preds, axis=0)

    #returns the loss and predictions
    return avg_loss, total_preds

In [ ]:
train_loss, _ = train()

Train:   0%|          | 0/62484 [00:00<?, ?it/s]

STEP 0: loss classifier: 0.006539450027048588
STEP 100: loss classifier: 0.005286253523081541
STEP 200: loss classifier: 0.00577233312651515
STEP 300: loss classifier: 0.006785879377275705
STEP 400: loss classifier: 0.006582504138350487
STEP 500: loss classifier: 0.0052382079884409904
STEP 600: loss classifier: 0.005633514374494553
STEP 700: loss classifier: 0.00442507304251194
STEP 800: loss classifier: 0.006413094699382782
STEP 900: loss classifier: 0.005706325173377991
STEP 1000: loss classifier: 0.005523086991161108
STEP 1100: loss classifier: 0.00558877270668745
STEP 1200: loss classifier: 0.005359854083508253
STEP 1300: loss classifier: 0.0065163918770849705
STEP 1400: loss classifier: 0.003570703323930502
STEP 1500: loss classifier: 0.007230360526591539
STEP 1600: loss classifier: 0.00527672516182065
STEP 1700: loss classifier: 0.00831750500947237
STEP 1800: loss classifier: 0.00614092405885458
STEP 1900: loss classifier: 0.004562295041978359
STEP 2000: loss classifier: 0.005175

Eval:   0%|          | 0/1634 [00:00<?, ?it/s]

loss contrastive :loss1  loss classifier: 0.005678244866430759
loss contrastive :loss1  loss classifier: 0.004505077842622995
Evaluate loss 0.005179616474293428
STEP 10000: loss classifier: 0.003268686356022954
STEP 10100: loss classifier: 0.0038966115098446608
STEP 10200: loss classifier: 0.004677766002714634
STEP 10300: loss classifier: 0.003893543966114521
STEP 10400: loss classifier: 0.005082374904304743
STEP 10500: loss classifier: 0.006504149176180363
STEP 10600: loss classifier: 0.004028094932436943
STEP 10700: loss classifier: 0.005414608400315046
STEP 10800: loss classifier: 0.005910307168960571
STEP 10900: loss classifier: 0.0037751924246549606
STEP 11000: loss classifier: 0.005412472411990166
STEP 11100: loss classifier: 0.004746235907077789
STEP 11200: loss classifier: 0.004026717506349087
STEP 11300: loss classifier: 0.00552744185552001
STEP 11400: loss classifier: 0.005490193609148264
STEP 11500: loss classifier: 0.005467909388244152
STEP 11600: loss classifier: 0.0041698

Eval:   0%|          | 0/1634 [00:00<?, ?it/s]

loss contrastive :loss1  loss classifier: 0.005192983895540237
loss contrastive :loss1  loss classifier: 0.0043900529853999615
Evaluate loss 0.004882681305916932
STEP 20000: loss classifier: 0.0031283122953027487
STEP 20100: loss classifier: 0.0062094018794596195
STEP 20200: loss classifier: 0.005252518225461245
STEP 20300: loss classifier: 0.004182801581919193
STEP 20400: loss classifier: 0.006056245416402817
STEP 20500: loss classifier: 0.004632787313312292
STEP 20600: loss classifier: 0.004803474526852369
STEP 20700: loss classifier: 0.004556267522275448
STEP 20800: loss classifier: 0.004039531573653221
STEP 20900: loss classifier: 0.0046360306441783905
STEP 21000: loss classifier: 0.0033480986021459103
STEP 21100: loss classifier: 0.0044764308258891106
STEP 21200: loss classifier: 0.0042644077911973
STEP 21300: loss classifier: 0.004757292568683624
STEP 21400: loss classifier: 0.004079400096088648
STEP 21500: loss classifier: 0.004243966657668352
STEP 21600: loss classifier: 0.0051

Eval:   0%|          | 0/1634 [00:00<?, ?it/s]

loss contrastive :loss1  loss classifier: 0.0051286290399730206
loss contrastive :loss1  loss classifier: 0.003897738875821233
Evaluate loss 0.004662337076862775
STEP 30000: loss classifier: 0.005140595603734255
STEP 30100: loss classifier: 0.0047587184235453606
STEP 30200: loss classifier: 0.005003014579415321
STEP 30300: loss classifier: 0.003529478795826435
STEP 30400: loss classifier: 0.003717755666002631
STEP 30500: loss classifier: 0.005274747498333454
STEP 30600: loss classifier: 0.0057831620797514915
STEP 30700: loss classifier: 0.004950101487338543
STEP 30800: loss classifier: 0.0038335053250193596
STEP 30900: loss classifier: 0.0038451431319117546
STEP 31000: loss classifier: 0.005153605714440346
STEP 31100: loss classifier: 0.006822861731052399
STEP 31200: loss classifier: 0.004872429650276899
STEP 31300: loss classifier: 0.004087185952812433
STEP 31400: loss classifier: 0.005991858895868063
STEP 31500: loss classifier: 0.004975345451384783
STEP 31600: loss classifier: 0.005

## Training

In [16]:
%%time
# set initial loss to infinite
import time
best_valid_loss = float('inf')
from tqdm.auto import tqdm

# empty lists to store training and validation loss of each epoch
train_losses=[]
valid_losses=[]

now=time.strftime('%Y-%m-%d %H:%M:%S', time.localtime())
#for each epoch
for epoch in tqdm(range(EPOCHS)):
     
    print('\n Epoch {:} / {:}'.format(epoch + 1, EPOCHS))
    
    #train model
    train_loss, _ = train()
    
    #evaluate model
    valid_loss, _, _ = evaluate(val_dataloader)
    
    #save the best model
    if valid_loss < best_valid_loss:
        best_valid_loss = valid_loss
        torch.save(model.state_dict(), f"./Baseline_abstract_model{now}.pt")
    
    # append training and validation loss
    train_losses.append(train_loss)
    valid_losses.append(valid_loss)
    


 Epoch 1 / 5


STEP 0: loss contrastive :0.0010610689641907811  loss classifier: 0.6774331331253052
STEP 50: loss contrastive :0.0010139003861695528  loss classifier: 0.38425585627555847
STEP 100: loss contrastive :0.0009989901445806026  loss classifier: 0.3425288498401642
STEP 150: loss contrastive :0.0009738221997395158  loss classifier: 0.312829852104187
STEP 200: loss contrastive :0.0009476971463300288  loss classifier: 0.2964755892753601
STEP 250: loss contrastive :0.0009283071267418563  loss classifier: 0.29871895909309387
STEP 300: loss contrastive :0.0009683565585874021  loss classifier: 0.29144302010536194
STEP 350: loss contrastive :0.0009231276926584542  loss classifier: 0.268029123544693
STEP 400: loss contrastive :0.0009250261355191469  loss classifier: 0.27633774280548096
STEP 450: loss contrastive :0.0009329313761554658  loss classifier: 0.292875736951828
STEP 500: loss contrastive :0.0009649590938352048  loss classifier: 0.28385740518569946
STEP 550: loss contrastive :0.00094192818505

loss contrastive :0.0008807076374068856  loss classifier: 0.21857139468193054
loss contrastive :0.0009065266931429505  loss classifier: 0.23401078581809998
loss contrastive :0.0009554822463542223  loss classifier: 0.24336905777454376
loss contrastive :0.0009160935296677053  loss classifier: 0.24640393257141113
loss contrastive :0.0009338076924905181  loss classifier: 0.25457191467285156
loss contrastive :0.0009518761653453112  loss classifier: 0.24289177358150482
loss contrastive :0.0009178954060189426  loss classifier: 0.21809642016887665

Evaluate loss 0.239400030033929

 Epoch 2 / 5


STEP 0: loss contrastive :0.0009442084701731801  loss classifier: 0.2308642864227295
STEP 50: loss contrastive :0.0009086896316148341  loss classifier: 0.24797853827476501
STEP 100: loss contrastive :0.0008787601254880428  loss classifier: 0.24488425254821777
STEP 150: loss contrastive :0.0009530895622447133  loss classifier: 0.2338808923959732
STEP 200: loss contrastive :0.000961251906119287  loss classifier: 0.2411338984966278
STEP 250: loss contrastive :0.000928273017052561  loss classifier: 0.2574857175350189
STEP 300: loss contrastive :0.0008967827307060361  loss classifier: 0.2313375324010849
STEP 350: loss contrastive :0.000887261179741472  loss classifier: 0.2302122563123703
STEP 400: loss contrastive :0.0009499300504103303  loss classifier: 0.23939299583435059
STEP 450: loss contrastive :0.0009526830399408937  loss classifier: 0.24924124777317047
STEP 500: loss contrastive :0.0009005170431919396  loss classifier: 0.23633794486522675
STEP 550: loss contrastive :0.00089170970022

loss contrastive :0.0008893839549273252  loss classifier: 0.20146054029464722
loss contrastive :0.0008835346670821309  loss classifier: 0.2135966420173645
loss contrastive :0.0009539992897771299  loss classifier: 0.22898778319358826
loss contrastive :0.0008915557991713285  loss classifier: 0.2200794667005539
loss contrastive :0.0009340487304143608  loss classifier: 0.23876436054706573
loss contrastive :0.0009146474185399711  loss classifier: 0.22410865128040314
loss contrastive :0.0009144307114183903  loss classifier: 0.20079663395881653

Evaluate loss 0.224818488742624

 Epoch 3 / 5


...

loss contrastive :0.0008923079585656524  loss classifier: 0.2057870328426361
loss contrastive :0.0008866586722433567  loss classifier: 0.21777421236038208
loss contrastive :0.0009292339673265815  loss classifier: 0.218560591340065
loss contrastive :0.0008858245564624667  loss classifier: 0.2149192988872528
loss contrastive :0.0009333193302154541  loss classifier: 0.24170167744159698
loss contrastive :0.0009262507664971054  loss classifier: 0.21084506809711456
loss contrastive :0.0009197652107104659  loss classifier: 0.18989942967891693

Evaluate loss 0.22155216634273528

 Epoch 4 / 5


STEP 0: loss contrastive :0.0009148622630164027  loss classifier: 0.18414299190044403
STEP 50: loss contrastive :0.000876838224940002  loss classifier: 0.16890180110931396
STEP 100: loss contrastive :0.0008993896190077066  loss classifier: 0.16637739539146423
STEP 150: loss contrastive :0.0009161243215203285  loss classifier: 0.1544165462255478
STEP 200: loss contrastive :0.0008454859489575028  loss classifier: 0.19036753475666046
STEP 250: loss contrastive :0.0008776029571890831  loss classifier: 0.17521975934505463
STEP 300: loss contrastive :0.000863123219460249  loss classifier: 0.15643495321273804
STEP 350: loss contrastive :0.0008465180872008204  loss classifier: 0.17740429937839508
STEP 400: loss contrastive :0.0008260203758254647  loss classifier: 0.17282404005527496
STEP 450: loss contrastive :0.0008817882044240832  loss classifier: 0.18425744771957397
STEP 500: loss contrastive :0.0008810520521365106  loss classifier: 0.17513351142406464
STEP 550: loss contrastive :0.00083236

loss contrastive :0.000891766743734479  loss classifier: 0.20928114652633667
loss contrastive :0.0008764481754042208  loss classifier: 0.2221929430961609
loss contrastive :0.0009337436640635133  loss classifier: 0.22371983528137207
loss contrastive :0.0008815022883936763  loss classifier: 0.21296896040439606
loss contrastive :0.0009235467296093702  loss classifier: 0.2502913475036621
loss contrastive :0.000922387174796313  loss classifier: 0.22078891098499298
loss contrastive :0.000905571854673326  loss classifier: 0.1916544884443283

Evaluate loss 0.2295128060238702

 Epoch 5 / 5


STEP 0: loss contrastive :0.0008306929375976324  loss classifier: 0.15690772235393524
STEP 50: loss contrastive :0.0008291496778838336  loss classifier: 0.15071210265159607
STEP 100: loss contrastive :0.0008183603640645742  loss classifier: 0.1562301069498062
STEP 150: loss contrastive :0.0008440852398052812  loss classifier: 0.14083687961101532
STEP 200: loss contrastive :0.000838672393001616  loss classifier: 0.1721179187297821
STEP 250: loss contrastive :0.0008693318231962621  loss classifier: 0.15354199707508087
STEP 300: loss contrastive :0.0008761153439991176  loss classifier: 0.1530335247516632
STEP 350: loss contrastive :0.0007590079912915826  loss classifier: 0.12554311752319336
STEP 400: loss contrastive :0.0008974550291895866  loss classifier: 0.15950292348861694
STEP 450: loss contrastive :0.0008219852461479604  loss classifier: 0.17826594412326813
STEP 500: loss contrastive :0.0008417931385338306  loss classifier: 0.14752903580665588
STEP 550: loss contrastive :0.000821863

loss contrastive :0.0008922904962673783  loss classifier: 0.22516782581806183
loss contrastive :0.0008801620570011437  loss classifier: 0.2342340350151062
loss contrastive :0.0009436492691747844  loss classifier: 0.2321746051311493
loss contrastive :0.0008874292252585292  loss classifier: 0.2229316383600235
loss contrastive :0.0009319459786638618  loss classifier: 0.2601303458213806
loss contrastive :0.0009146207594312727  loss classifier: 0.2369941771030426
loss contrastive :0.0009042565943673253  loss classifier: 0.2024374008178711

Evaluate loss 0.24233244167906898

CPU times: user 55min 33s, sys: 6min 10s, total: 1h 1min 44s
Wall time: 56min 15s


In [16]:
%%time
# set initial loss to infinite
import time
best_valid_loss = float('inf')
from tqdm.auto import tqdm

# empty lists to store training and validation loss of each epoch
train_losses=[]
valid_losses=[]

now=time.strftime('%Y-%m-%d %H:%M:%S', time.localtime())
#for each epoch
for epoch in tqdm(range(EPOCHS)):
     
    print('\n Epoch {:} / {:}'.format(epoch + 1, EPOCHS))
    
    #train model
    train_loss, _ = train()
    
    #evaluate model
    valid_loss, _, _ = evaluate(val_dataloader)
    
    #save the best model
    if valid_loss < best_valid_loss:
        best_valid_loss = valid_loss
        torch.save(model.state_dict(), f"./Baseline_abstract_model{now}.pt")
    
    # append training and validation loss
    train_losses.append(train_loss)
    valid_losses.append(valid_loss)
    


 Epoch 1 / 5


STEP 0: loss contrastive :0.0011736995074898005  loss classifier: 0.6784937977790833
STEP 50: loss contrastive :0.0010244036093354225  loss classifier: 0.38525381684303284
STEP 100: loss contrastive :0.0010459651239216328  loss classifier: 0.34234097599983215
STEP 150: loss contrastive :0.0010059644700959325  loss classifier: 0.31425362825393677
STEP 200: loss contrastive :0.0009743745322339237  loss classifier: 0.29644834995269775
STEP 250: loss contrastive :0.000911412644200027  loss classifier: 0.3009869456291199
STEP 300: loss contrastive :0.000994911533780396  loss classifier: 0.2920014560222626
STEP 350: loss contrastive :0.0009435606189072132  loss classifier: 0.27130696177482605
STEP 550: loss contrastive :0.0009513312834315002  loss classifier: 0.2624566853046417
STEP 600: loss contrastive :0.0008490897598676383  loss classifier: 0.2606485188007355
STEP 650: loss contrastive :0.0009443267481401563  loss classifier: 0.2466069906949997
STEP 700: loss contrastive :0.0009472237434

loss contrastive :0.00086324627045542  loss classifier: 0.22190608084201813
loss contrastive :0.0008816148620098829  loss classifier: 0.23421892523765564
loss contrastive :0.0010400454048067331  loss classifier: 0.2457604855298996
loss contrastive :0.0009191778372041881  loss classifier: 0.2457519769668579
loss contrastive :0.0009707127464935184  loss classifier: 0.25364264845848083
loss contrastive :0.0009471081430092454  loss classifier: 0.23740224540233612
loss contrastive :0.000926076085306704  loss classifier: 0.22345615923404694

Evaluate loss 0.23963707110711507

 Epoch 2 / 5


STEP 0: loss contrastive :0.0009488399373367429  loss classifier: 0.23232369124889374
STEP 50: loss contrastive :0.0008958855760283768  loss classifier: 0.253220796585083
STEP 100: loss contrastive :0.0008832403691485524  loss classifier: 0.2489113211631775
STEP 150: loss contrastive :0.0009393055224791169  loss classifier: 0.23295415937900543
STEP 200: loss contrastive :0.0009999671019613743  loss classifier: 0.2376948744058609
STEP 250: loss contrastive :0.0009627904510125518  loss classifier: 0.25691667199134827
STEP 300: loss contrastive :0.0008979059057310224  loss classifier: 0.2328859567642212
STEP 350: loss contrastive :0.0008929578470997512  loss classifier: 0.22803781926631927
STEP 400: loss contrastive :0.0010320055298507214  loss classifier: 0.24067842960357666
STEP 450: loss contrastive :0.0009719012305140495  loss classifier: 0.2511763274669647
STEP 500: loss contrastive :0.0009282968239858747  loss classifier: 0.24083101749420166
STEP 550: loss contrastive :0.00087546568

loss contrastive :0.0008599188877269626  loss classifier: 0.2006521224975586
loss contrastive :0.0008460755925625563  loss classifier: 0.2135000228881836
loss contrastive :0.000979618402197957  loss classifier: 0.22505564987659454
loss contrastive :0.0008922427077777684  loss classifier: 0.22128352522850037
loss contrastive :0.0010183851700276136  loss classifier: 0.24932970106601715
loss contrastive :0.000922065693885088  loss classifier: 0.22875399887561798
loss contrastive :0.0009104629280045629  loss classifier: 0.19750604033470154

Evaluate loss 0.22413069563252586

 Epoch 3 / 5


...

loss contrastive :0.0008725500083528459  loss classifier: 0.20578263700008392
loss contrastive :0.000910047092474997  loss classifier: 0.21890391409397125
loss contrastive :0.0009645494865253568  loss classifier: 0.2212047576904297
loss contrastive :0.0009276264463551342  loss classifier: 0.2225142866373062
loss contrastive :0.0010248979087918997  loss classifier: 0.24798201024532318
loss contrastive :0.0009418590925633907  loss classifier: 0.21279238164424896
loss contrastive :0.0009347681188955903  loss classifier: 0.18822413682937622

Evaluate loss 0.22210357636213302

 Epoch 4 / 5


STEP 0: loss contrastive :0.000942211365327239  loss classifier: 0.18686386942863464
STEP 50: loss contrastive :0.0008771411376073956  loss classifier: 0.168535053730011
STEP 100: loss contrastive :0.0009232746670022607  loss classifier: 0.17536720633506775
STEP 150: loss contrastive :0.0009367841412313282  loss classifier: 0.15939436852931976
STEP 200: loss contrastive :0.0008358241757377982  loss classifier: 0.20461532473564148
STEP 250: loss contrastive :0.0008678673766553402  loss classifier: 0.176971897482872
STEP 300: loss contrastive :0.000818393484223634  loss classifier: 0.15583422780036926
STEP 350: loss contrastive :0.0008111900533549488  loss classifier: 0.176048144698143
STEP 400: loss contrastive :0.0007947029080241919  loss classifier: 0.17967219650745392
STEP 450: loss contrastive :0.0008485829457640648  loss classifier: 0.18107275664806366
STEP 500: loss contrastive :0.000850270502269268  loss classifier: 0.17511844635009766
STEP 550: loss contrastive :0.00081645464524

loss contrastive :0.0008937601232901216  loss classifier: 0.2024717777967453
loss contrastive :0.0009234985336661339  loss classifier: 0.22096052765846252
loss contrastive :0.0009677024208940566  loss classifier: 0.22529539465904236
loss contrastive :0.0009381117415614426  loss classifier: 0.21840733289718628
loss contrastive :0.001024402561597526  loss classifier: 0.25764214992523193
loss contrastive :0.000994328293018043  loss classifier: 0.2228819578886032
loss contrastive :0.000939505931455642  loss classifier: 0.1843181699514389

Evaluate loss 0.22800866003547396

 Epoch 5 / 5


STEP 0: loss contrastive :0.0007900340715423226  loss classifier: 0.15494336187839508
STEP 50: loss contrastive :0.0008160884608514607  loss classifier: 0.15528662502765656
STEP 100: loss contrastive :0.0008477926021441817  loss classifier: 0.16335934400558472
STEP 150: loss contrastive :0.0008415866177529097  loss classifier: 0.14178107678890228
STEP 200: loss contrastive :0.0008564976742491126  loss classifier: 0.18279756605625153
STEP 250: loss contrastive :0.0008687674999237061  loss classifier: 0.1595715433359146
STEP 300: loss contrastive :0.0009081882890313864  loss classifier: 0.16121934354305267
STEP 350: loss contrastive :0.0006919500301592052  loss classifier: 0.13182401657104492
STEP 400: loss contrastive :0.0008912598132155836  loss classifier: 0.15460121631622314
STEP 450: loss contrastive :0.0007779154693707824  loss classifier: 0.1831590086221695
STEP 500: loss contrastive :0.0008356108446605504  loss classifier: 0.16439509391784668
STEP 550: loss contrastive :0.0008153

loss contrastive :0.0009505008347332478  loss classifier: 0.21422308683395386
loss contrastive :0.000962299294769764  loss classifier: 0.22596541047096252
loss contrastive :0.0009981528855860233  loss classifier: 0.2330978512763977
loss contrastive :0.000995664857327938  loss classifier: 0.22896067798137665
loss contrastive :0.00106520252302289  loss classifier: 0.26746976375579834
loss contrastive :0.0010346363997086883  loss classifier: 0.23680955171585083
loss contrastive :0.0009839585982263088  loss classifier: 0.19763624668121338

Evaluate loss 0.24067499275718415

CPU times: user 1h 10min 51s, sys: 17min 12s, total: 1h 28min 4s
Wall time: 1h 22min 35s


In [57]:
import numpy as np
import pandas as pd
from sklearn.metrics import (accuracy_score, precision_score, recall_score, f1_score, roc_auc_score)


true = np.array(total_labels)
pred = np.array(total_preds>0.5)

dic = {
    "Accuracy" : accuracy_score(true,pred),
    "Precision-micro" : precision_score(true,pred,average='micro'),
    "Precision-macro" : precision_score(true,pred,average='macro'),
    "recall-micro" : recall_score(true,pred,average='micro'),
    "recall-macro" : recall_score(true,pred,average='macro'),
    "f1_micro" : f1_score(true,pred,average='micro'),
    "f1-macro" : f1_score(true,pred,average='macro')
}

/root/miniconda3/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/root/miniconda3/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/root/miniconda3/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1580: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no true nor predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, "true nor predicted", "F-score is", len(true_sum))


In [58]:
dic

{'Accuracy': 0.40685096153846156,
 'Precision-micro': 0.8202037351443124,
 'Precision-macro': 0.43446532160346085,
 'recall-micro': 0.5257945145842403,
 'recall-macro': 0.26941306850665814,
 'f1_micro': 0.6408011672635628,
 'f1-macro': 0.3154646223318294}

In [59]:
torch.save(model.state_dict(),"./model/CoPatE.pt")

## batch_size = 8时的模型测试

In [ ]:
model.load_state_dict(torch.load("./model/Classfication_Baseline_claims_model2022-02-22 01:57:06.pt"))

In [27]:
test_df = pd.read_feather("2021-sample-5000_retrieval_two.feather")

In [31]:
test_df.claims

0       [A hanging pocket for point-of-sale display, t...
1       [A method for growing a crystal, comprising:we...
2       [A device for monitoring usage of a toothbrush...
3       [A steerable catheter robotic system, comprisi...
4       [A battery-powered device comprising:a power i...
                              ...                        
4995    [A method of making a carbon nanotube composit...
4996    [A storage system comprising a first storage b...
4997    [A shoulder implant system comprising:a glenoi...
4998    [A noise cancelling soundbar, comprising:one o...
4999    [A method for transforming an algal chloroplas...
Name: claims, Length: 5000, dtype: object

In [28]:
test_dataset = PatentDataset(
  test_df,
  tokenizer,
  max_token_len=MAX_TOKEN_COUNT,
  test = True
)
test_dataloader = DataLoader(test_dataset, batch_size=32, shuffle=False,drop_last=True)

avg_loss, total_preds, total_labels = evaluate(test_dataloader)


Evaluating...


Eval:   0%|          | 0/156 [00:00<?, ?it/s]

loss contrastive :loss1  loss classifier: 0.0064818705432116985
Evaluate loss 0.0064623038514326206
